In [1]:
!pip install langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
from langchain.agents import initialize_agent, Tool # Tool is a rapper function
from langchain.tools import tool # tool is decarator
from langchain.agents.agent_types import AgentType # used to declare the agent type that we are going to work on
from langchain_google_genai import ChatGoogleGenerativeAI
import imaplib, smtplib, email
from email.mime.text import MIMEText # minetext tt is used to create plain text or HTML email message bodies
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "UPLOAD YOUR GOOGLE API KEY HERE"
EMAIL = "UPLOAD YOUR EMAIL HERE"
APP_PASSWORD = "UPLOAD YOUR APP PASSWORD HERE"

In [14]:
@tool
def read_gmail(query : str = "") -> str:
  """Reads the latest unread Gmail message."""
  mail = imaplib.IMAP4_SSL("imap.gmail.com")
  mail.login(EMAIL, APP_PASSWORD)
  mail.select("inbox")
  status, data = mail.search(None, "UNSEEN")
  ids = data[0].split()
  if not ids:
    return "No unread emails found"
  latest_id = ids[-1]
  status, msg_data = mail.fetch(latest_id, "(RFC822)")
  msg = email.message_from_bytes(msg_data[0][1])
  subject = msg["Subject"]
  sender = msg["From"]
  body = ""

  if msg.is_multipart():
    for part in msg.walk():
      if part.get_content_type() == 'text/plain':
        body += part.get_payload(decode=True).decode()
  else:
    body  = msg.get_payload(decode=True).decode()
  return f"from: {sender}\nSubject : {subject}\nBody\n{body.strip()}"


In [27]:
@tool
def send_gmail_reply(details : str) -> str:
  """Sends a reply to an email. Format : 'to|subject|message. """
  try:
    to_email, subject, body = details.split("|",2)
    msg = MIMEText(body)
    msg["Subject"] = "Re: "  + subject
    msg["From"] = EMAIL
    msg["To"] = to_email
    server = smtplib.SMTP_SSL("smtp.gmail.com",465)
    server.login(EMAIL, APP_PASSWORD)
    server.sendmail(EMAIL, to_email, msg.as_string())
    server.quit()

    return "reply send succesfully"
  except exception as e:
    return f"Failed to send : {str(e)}"

In [28]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
tools = [read_gmail, send_gmail_reply]

In [29]:
from pickle import TRUE
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [31]:
prompt = """
  Read the latest unread email. If it's a customer query , write a polite reply and sent it. To send reply, format must be : to|subject|message
"""
agent.run(prompt)



> Entering new AgentExecutor chain...
Thought: I need to read the latest unread Gmail message to see if it's a customer query.
Action: read_gmail
Action Input: ''
Observation: from: ARIKARAN R <arikaran257@gmail.com>
Subject : Damaged headset
Body
Dear Customer Support,

I hope this message finds you well. I am writing to report an issue with a
headset I purchased from your store (Order ID: #123456).

Unfortunately, the headset arrived damaged and is not functioning properly.
I have tried basic troubleshooting, but the issue persists.

I kindly request a replacement or refund for the faulty product. Please let
me know if any further details or photos are required.

Looking forward to your prompt assistance. Kind regards
Thought:Question: Read the latest unread email. If it's a customer query , write a polite reply and sent it. To send reply, format must be : to|subject|message

Thought: I need to read the latest unread Gmail message to see if it's a customer query.
Action: read_gmail

'A reply has been sent to the customer regarding their damaged headset.'